In [1]:
import pandas as pd

df = pd.read_excel("Intern Excel Exercise DB.xlsx",sheet_name = "DB Exercise")

print(df.head(5))


  Processor Brand OEM Brand Processor Family  Year  Quarter     Month  \
0     Competition     OEM A          AMD A10  2019  Q1 2019  Jan 2019   
1     Competition     OEM A          AMD A10  2019  Q2 2019  May 2019   
2     Competition     OEM A          AMD A10  2019  Q3 2019  Sep 2019   
3     Competition     OEM A          AMD A10  2019  Q4 2019  Oct 2019   
4     Competition     OEM A          AMD A10  2020  Q3 2020  Jul 2020   

  Form Factor Parent Processor Generation  Sales Out Units  System Revenue $  
0           Notebook        Not Specified                3           1350.24  
1           Notebook        Not Specified                1            831.60  
2           Notebook        Not Specified                1            311.00  
3           Notebook        Not Specified                1            342.30  
4           Notebook        Not Specified                0            464.40  


In [2]:
##Question 1
sales_2020 = df[df['Year'] == 2020]['Sales Out Units'].sum()

sales_2019 = df[df['Year'] == 2019]['Sales Out Units'].sum()

growth_percentage = ((sales_2020 - sales_2019) / sales_2019) * 100

print(f"The % of growth of Sales Out Units in 2020 vs 2019 is: {growth_percentage:.2f}%")

The % of growth of Sales Out Units in 2020 vs 2019 is: 46.93%


In [3]:
##Question 2
##I found different results compared to the Excel sheet.

# Filter the data to include only the months of 2020 and 2019
df['Year'] = pd.to_numeric(df['Year'])  # Converting the 'Year' column to numeric
df = df[(df['Year'] == 2019) | (df['Year'] == 2020)]

# For each month in 2020, find the corresponding month in 2019
def find_corresponding_month(row):
    month_year = row['Month'].split(' ')
    month, year = month_year[0], int(month_year[1])
    corresponding_year = year - 1
    corresponding_month = f"{month} {corresponding_year}"
    return corresponding_month

df['Corresponding Month'] = df.apply(find_corresponding_month, axis=1)

def calculate_growth(row):
    current_month_sales = df[(df['Month'] == row['Month']) & (df['Year'] == 2020)]['Sales Out Units'].sum()
    corresponding_month_sales = df[df['Month'] == row['Corresponding Month']]['Sales Out Units'].sum()
    
    # Check if the corresponding_month_sales is not zero to avoid division by zero
    if corresponding_month_sales != 0:
        growth = ((current_month_sales - corresponding_month_sales) / corresponding_month_sales) * 100
    else:
        growth = 0  
        
    return growth

df['Sales Growth %'] = df.apply(calculate_growth, axis=1)

# Find the month with the highest sales growth in 2020
max_growth_month_2020 = df[df['Year'] == 2020].loc[df['Sales Growth %'].idxmax()]

print("Month with the highest notebook sales growth in 2020:")
print("Month:", max_growth_month_2020['Month'])
print("Growth compared to the same month in 2019:", max_growth_month_2020['Sales Growth %'], "%")



Month with the highest notebook sales growth in 2020:
Month: May 2020
Growth compared to the same month in 2019: 146.85992400631858 %


In [4]:
## Question 3
# Filter the data to include only the Notebook segment and Intel processors
notebook_df = df[(df['Form Factor Parent'] == 'Notebook') & (df['Processor Brand'] == 'Intel')]

# Group the data by month and calculate the total sales units for Intel processors and overall sales units for each month
monthly_sales = notebook_df.groupby('Month').agg({'Sales Out Units': 'sum'})
monthly_total_sales = df.groupby('Month').agg({'Sales Out Units': 'sum'})

# Calculate the Intel Market Share for each month
monthly_sales['Intel Market Share %'] = (monthly_sales['Sales Out Units'] / monthly_total_sales['Sales Out Units']) * 100

# Find the month with the highest Intel Market Share
month_with_highest_market_share = monthly_sales['Intel Market Share %'].idxmax()
highest_market_share = monthly_sales.loc[month_with_highest_market_share, 'Intel Market Share %']

print("Month with the highest % Intel Market Share in the Notebook segment:")
print("Month:", month_with_highest_market_share)
print("Intel Market Share:", highest_market_share, "%")


Month with the highest % Intel Market Share in the Notebook segment:
Month: Jan 2019
Intel Market Share: 75.83892617449665 %


In [5]:
## Question 4
# Filter the data to include only the Desktop segment and the year 2020
desktop_2020_df = df[(df['Form Factor Parent'] == 'Desktop') & (df['Year'] == 2020)]

# Group the data by OEM and calculate the total System Revenue for each OEM in 2020
oem_system_revenue_2020 = desktop_2020_df.groupby('OEM Brand')['System Revenue $'].sum()

# Find the OEM with the highest System Revenue
biggest_market_share_oem = oem_system_revenue_2020.idxmax()
biggest_market_share_system_revenue = oem_system_revenue_2020.max()

print("OEM with the biggest share of the market in the Desktop segment in 2020:")
print("OEM Brand:", biggest_market_share_oem)
print("System Revenue:", biggest_market_share_system_revenue)


OEM with the biggest share of the market in the Desktop segment in 2020:
OEM Brand: OEM C
System Revenue: 7305596.88


In [6]:
## Question 5
##I found different results compared to the Excel sheet

# Filter the data to include only the year 2020
df_2020 = df[df['Year'] == 2020].copy()

# Calculate the ASP for each Processor Family and assign to the original DataFrame using .loc
df_2020['ASP'] = df_2020['System Revenue $'] / df_2020['Sales Out Units']
df.loc[df['Year'] == 2020, 'ASP'] = df_2020['ASP']

# Find the Processor Family with the highest ASP
highest_asp_family = df_2020.groupby('Processor Family')['ASP'].mean().idxmax()
highest_asp_value = df_2020.groupby('Processor Family')['ASP'].mean().max()

print("Processor Family with the highest ASP in 2020:")
print("Processor Family:", highest_asp_family)
print("Average Selling Price (ASP):", highest_asp_value)


Processor Family with the highest ASP in 2020:
Processor Family: AMD Ryzen 9
Average Selling Price (ASP): 1663.8338528082907
